In [ ]:
# IMPORTING LIBRARIES, PACKAGES, SETTING WORKING DIRECTORIES AND DEFINING VARIABLES

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
import pafy
import os
import np_utils
from sklearn.cross_validation import train_test_split

Main_Folder='/Users/gautamroy/Documents/My Python Package/Python OpenCV'
Training_Folder='/Users/gautamroy/Documents/My Python Package/Python OpenCV/Training_set'
Validation_Folder='/Users/gautamroy/Documents/My Python Package/Python OpenCV/Validation_set'

SPLIT_RATIO = 0.2


In [ ]:
# SPLITTING VIDEO FILES INTO TRAINING AND VALIDATION SETS

os.chdir(Main_Folder)
YT_URL_Dataset=pd.read_csv('Dataset_VideoURL_Category.csv')
X_train_videos,X_valid_videos,y_train_videos,y_valid_videos=train_test_split(YT_URL_Dataset['YouTube_URL'],YT_URL_Dataset['Category'],test_size=SPLIT_RATIO,random_state=0)
X_train_videos=np.array(X_train_videos)
X_valid_videos=np.array(X_valid_videos)
y_train_videos=np.array(y_train_videos)
y_valid_videos=np.array(y_valid_videos)


In [ ]:
# EXTRACTING FRAMES FROM VIDEO FILES

def FRAME_EXTRACTION(x_dataset,y_dataset,dstype):
    
    Train_Frame_set=pd.DataFrame(columns=['FrameID','Category'])
    Validation_Frame_set=pd.DataFrame(columns=['FrameID','Category'])
    
    for i in range(x_dataset.shape[0]):
        
        print('Processing Video:',(i+1),x_dataset[i])
        video = pafy.new(x_dataset[i])
        best = video.getbest(preftype="mp4")
        capture = cv2.VideoCapture()
        capture.open(best.url)
        #fps = np.round(capture.get(cv2.CAP_PROP_FPS))
        success,image = capture.read()
        count = 0
        
        if (dstype=='Train'):
            os.chdir(Training_Folder)
            while (capture.isOpened() & success):
                if (count%300 == 0):
                    cv2.imwrite("Video%s_frame%d.jpg" % ((i+1),count), image)
                    Train_Frame_set = Train_Frame_set.append({'FrameID':"Video%d_frame%d.jpg" % ((i+1),count),'Category':y_dataset[i]},ignore_index=True)
                success,image = capture.read()
                count += 1  
        
        else:
            os.chdir(Validation_Folder)
            while (capture.isOpened() & success):
                if (count%300 == 0):
                    cv2.imwrite("Video%s_frame%d.jpg" % ((i+1),count), image)
                    Validation_Frame_set = Validation_Frame_set.append({'FrameID':"Video%d_frame%d.jpg" % ((i+1),count),'Category':y_dataset[i]},ignore_index=True)
                success,image = capture.read()
                count += 1    
        
    if (dstype=='Train'):
        return Train_Frame_set
    else:
        return Validation_Frame_set
    
    capture.release()


In [ ]:
# GENERATING CSV FILE WITH FRAME NAMES AND CATAGORY NAMES

os.chdir(Main_Folder)

Train_Frame_set = FRAME_EXTRACTION(X_train_videos,y_train_videos,'Train')
Train_Frame_set.to_csv('Train_Frame_set.csv')

Validation_Frame_set = FRAME_EXTRACTION(X_valid_videos,y_valid_videos,'Valid')
Validation_Frame_set.to_csv('Validation_Frame_set.csv')
